In [1]:
import numpy as np
import veloxchem as vlx

In [2]:
h2o_xyz = """3

O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""
molecule = vlx.Molecule.read_xyz_string(h2o_xyz)
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz", ostream=None)

In [3]:
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_drv.xcfun = "b3lyp"
scf_results = scf_drv.compute(molecule, basis)

In [5]:
D = scf_results["D_alpha"] + scf_results["D_beta"]

In [6]:
grid_level = scf_drv.grid_level
if grid_level is None:
    grid_level = vlx.dftutils.get_default_grid_level(scf_drv.xcfun)

grid_drv = vlx.GridDriver()
grid_drv.set_level(grid_level)
molgrid = grid_drv.generate(molecule)

# grid point weights
w_g = molgrid.w_to_numpy()

# grid point z-coordinates
z_g = molgrid.z_to_numpy()

In [7]:
xc_drv = vlx.XCIntegrator()

# generate AOs on the grid points
chi_g = xc_drv.compute_gto_values(molecule, basis, molgrid)

# determine the density on the grid points
G = np.einsum("ab,bg->ag", D, chi_g)
n_g = np.einsum("ag,ag->g", chi_g, G)

print(f"Dipole moment: {-1.0 * np.dot(w_g, z_g * n_g) : 12.8f}")

Dipole moment:  -1.44687595


In [8]:
dipole_drv = vlx.ElectricDipoleIntegralsDriver()
dipole_mats = dipole_drv.compute(molecule, basis)

mu_z = -1.0 * dipole_mats.z_to_numpy()

In [9]:
print(f"Dipole moment: {np.einsum('ab,ab->', D, mu_z) : 12.8f}")

Dipole moment:  -1.44687507
